In [1]:
import os
# Set the CUDA_VISIBLE_DEVICES environment variable
#export CUDA_VISIBLE_DEVICES=0
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

/data/juhwan/nlp-task-translation-en-ko/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# https://huggingface.co/Helsinki-NLP/opus-mt-ko-en

model_checkpoint = "Helsinki-NLP/opus-mt-ko-en"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors="pt")
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

/data/juhwan/nlp-task-translation-en-ko/.venv/lib/python3.8/site-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [4]:
import torch
torch.__version__

'2.0.0+cu117'

In [5]:
# dataset
from datasets import load_dataset

dataset = load_dataset('json', data_files='./ai_hub/1.Training/원천데이터/일상생활및구어체_한영_train_set.json', field='data')

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 799.07it/s]


Dataset json downloaded and prepared to /home/juhwan/.cache/huggingface/datasets/json/default-3c8e71bae241100f/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 26.76it/s]


In [6]:
split_datasets = dataset["train"].train_test_split(train_size=0.9, seed=20)
split_datasets["validation"] = split_datasets.pop("test")

In [7]:
# declare preprocessing code 

max_input_length = 128
max_target_length = 128

def preprocess_function(examples):
    inputs = examples['ko']
    targets = examples['en']

    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # 타겟을 위한 토크나이저 셋업
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs            

In [8]:
tokenized_datasets = split_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=split_datasets["train"].column_names,
)

Map:   0%|          | 0/1080000 [00:00<?, ? examples/s]/data/juhwan/nlp-task-translation-en-ko/.venv/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:3596: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [9]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [11]:
from datasets import load_metric
metric = load_metric("sacrebleu")

In [12]:
# 코드 자세히봐야해.. 왜 -100을 제거하고 그런거 알아봐야한다.
import numpy as np

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # 모델이 예측 로짓(logits)외에 다른 것을 리턴하는 경우.
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # -100은 건너뛴다.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # 단순 후처리
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

In [13]:
from transformers import Seq2SeqTrainingArguments

args = Seq2SeqTrainingArguments(
    f"finetuned-aihub-ko-to-en",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
)

In [14]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [15]:
# 학습하기전 초기모델 점수
trainer.evaluate(max_length=max_target_length)

{'eval_loss': 1.700509786605835,
 'eval_bleu': 19.12530451763208,
 'eval_runtime': 7044.9731,
 'eval_samples_per_second': 17.033,
 'eval_steps_per_second': 0.266}

In [16]:
trainer.train()

/data/juhwan/nlp-task-translation-en-ko/.venv/lib/python3.8/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


TrainOutput(global_step=101250, training_loss=0.8536538732458044, metrics={'train_runtime': 8704.5092, 'train_samples_per_second': 372.221, 'train_steps_per_second': 11.632, 'total_flos': 2.768806582891315e+16, 'train_loss': 0.8536538732458044, 'epoch': 3.0})

In [17]:
trainer.evaluate(max_length=max_target_length)

{'eval_loss': 0.7568225264549255,
 'eval_bleu': 52.00317662422721,
 'eval_runtime': 3684.4244,
 'eval_samples_per_second': 32.57,
 'eval_steps_per_second': 0.509,
 'epoch': 3.0}